# Download Data - GCFS

In [1]:
import autoroot
import ee
import xarray as xr
import cdsapi
import climetlab as cml
from pathlib import Path
import pprint
from bayesevt._src.data.era5.ops import parse_single_levels, parse_pressure_levels, parse_all_variables
from bayesevt._src.models.earth2mip import VARIABLES
from bayesevt._src.data.era5.variables import VARIABLES_SURFACE

%load_ext autoreload
%autoreload 2

In [2]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar
# client = Client(address="tcp://127.0.0.1:37059")
# client

## Single Levels

## Case I: Explicit Variables Names

In this case, we have explicit variable names from the model. 
Typically, there is some sense with how these work.

**Single Level Variables**. We have explicit surface variables which have a specific name.

**Pressure Level Variables**. We have some explicit pressure level variables with a specific name as a *single character* and an associated pressure level.

In [3]:
# channel_names = VARIABLES["fcnv2_sm"]
channels = list(set(VARIABLES["pangu"] + VARIABLES["fcnv2_sm"]))
# channels = VARIABLES["pangu"]

In [4]:
# parse all levels
all_variables = parse_all_variables(channels)
assert list(map(lambda x: x.name, all_variables)) == channels

In [5]:
# get surface variables
vars_surface = set(filter(lambda x: x.name in VARIABLES_SURFACE, all_variables))
vars_surface = set(map(lambda x: x.era5_name, vars_surface))

# get pressure variables
vars_pressure = set(filter(lambda x: x.name not in VARIABLES_SURFACE, all_variables))
unique_levels = set(map(lambda x: x.level, vars_pressure))
vars_pressure = set(map(lambda x: x.era5_name, vars_pressure))

# concatenate all variables
vars_all = list(vars_surface) + list(vars_pressure)

## Cloud Platform

#### Test Case

> this is a simple test case for the doc strings

In [6]:
cloud_dir = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"

In [7]:
%%time
ds = xr.open_zarr(cloud_dir)
ds

CPU times: user 1min 7s, sys: 23.9 s, total: 1min 31s
Wall time: 1min 25s


<xarray.Dataset> Size: 3PB
Dimensions:                                                          (
                                                                      time: 1085496,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * latitude                                                         (latitude) float32 3kB ...
  * level                                                            (level) int64 296B ...
  * longitude                                                        (longitude) float32 6kB ...
  * time                                                             (time) datetime64[ns] 9MB ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_directional_width_for_wind_waves                   (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_kurtosis                                           (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_peakedness                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_skewness                                           (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    zero_degree_level                                                (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

**Preprocessing**

* select variables
* select time period
* select levels

In [8]:
ivars = list(filter(lambda x: x != "relative_humidity", vars_all))

In [9]:
with ProgressBar():
    # subset variables
    ds_subset = ds[ivars]
    # select event
    ds_subset = ds_subset.sel(time=slice("2018-07-20", "2018-08-10"))
    # select levels
    ds_subset = ds_subset.sel(level=list(unique_levels))
    # resample to 1D
    ds_subset = ds_subset.resample(time="1D").mean()
    # run parallel computation
    ds_subset = ds_subset.compute()

[#############################           ] | 72% Completed | 29m 47ss

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed | 44m 1s


In [10]:
%%time
save_name = "reanalysis_20180715_20180830_ics"
ds_subset.to_netcdf(f"/pool/proyectos/CLINT/sa4attrs/data/raw/events/{save_name}.nc")

CPU times: user 202 ms, sys: 3.78 s, total: 3.98 s
Wall time: 11.2 s


In [13]:
ds_subset_ = xr.open_dataset(f"/pool/proyectos/CLINT/sa4attrs/data/raw/events/{save_name}.nc")
ds_subset_

<xarray.Dataset> Size: 7GB
Dimensions:                    (time: 22, latitude: 721, longitude: 1440,
                                level: 13)
Coordinates:
  * latitude                   (latitude) float32 3kB 90.0 89.75 ... -90.0
  * level                      (level) int64 104B 100 200 1000 ... 250 700 925
  * longitude                  (longitude) float32 6kB 0.0 0.25 ... 359.5 359.8
  * time                       (time) datetime64[ns] 176B 2018-07-20 ... 2018...
Data variables: (12/13)
    10m_v_component_of_wind    (time, latitude, longitude) float32 91MB ...
    100m_v_component_of_wind   (time, latitude, longitude) float32 91MB ...
    surface_pressure           (time, latitude, longitude) float32 91MB ...
    10m_u_component_of_wind    (time, latitude, longitude) float32 91MB ...
    total_column_water_vapour  (time, latitude, longitude) float32 91MB ...
    2m_temperature             (time, latitude, longitude) float32 91MB ...
    ...                         ...
    100m_u_component_of_wind   (time, latitude, longitude) float32 91MB ...
    specific_humidity          (time, level, latitude, longitude) float32 1GB ...
    temperature                (time, level, latitude, longitude) float32 1GB ...
    geopotential               (time, level, latitude, longitude) float32 1GB ...
    u_component_of_wind        (time, level, latitude, longitude) float32 1GB ...
    v_component_of_wind        (time, level, latitude, longitude) float32 1GB ...

In [14]:
from bayesevt._src.geoprocessing.physics import CalculateRelativeHumidity

In [17]:
ds_subset_["relative_humidity"] = CalculateRelativeHumidity().calculate(ds_subset_)

In [18]:
save_name = "reanalysis_20180715_20180830_ics_v2"
ds_subset_.to_netcdf(f"/pool/proyectos/CLINT/sa4attrs/data/raw/events/{save_name}.nc")

In [11]:
!ls

0.2-reanalysis-slim.ipynb	     regions.ipynb    Untitled.ipynb
0.3-reanalysis-earth2mip-cds.ipynb   surface.nc       weatherbench_era5.ipynb
0.4-reanalysis-earth2mip-gcfs.ipynb  temp.grib
reanalysis.ipynb		     Untitled1.ipynb
